In [2]:
!pip install torch torchvision torchaudio  # Para PyTorch



[notice] A new release of pip is available: 25.2 -> 25.3
[notice] To update, run: pip install --upgrade pip


In [3]:
!pip install transformers datasets


[notice] A new release of pip is available: 25.2 -> 25.3
[notice] To update, run: pip install --upgrade pip


In [4]:
!pip install accelerate


[notice] A new release of pip is available: 25.2 -> 25.3
[notice] To update, run: pip install --upgrade pip


In [5]:
#warnings control
import warnings
warnings.filterwarnings('ignore')

In [6]:
# import the required classes
from transformers import AutoModelForCausalLM, AutoTokenizer, pipeline

In [21]:
# Load model and tokenizer
model_id = "microsoft/Phi-3-mini-4k-instruct"

tokenizer = AutoTokenizer.from_pretrained(model_id)

model = AutoModelForCausalLM.from_pretrained(
    model_id,
    device_map="cpu",
    dtype="auto", # Actualizado de torch_dtype
    # trust_remote_code=True,
    attn_implementation="eager" # Para evitar el aviso de flash-attention
)

Loading checkpoint shards:   0%|          | 0/2 [00:00<?, ?it/s]

In [22]:
# Create a pipeline
generator = pipeline(
    "text-generation",
    model=model,
    tokenizer=tokenizer,
    return_full_text=False, # False means to not include the prompt text in the returned text
    max_new_tokens=50, 
    do_sample=False, # no randomness in the generated text
)

Device set to use cpu


In [33]:
prompt = "Write an email apologizing to Conrado for the tragic gardening mishap. Explain how it happened and who was involved in it. Someone died. And a dinosaur appeared "

output = generator(prompt)

#lista de diccionarios es básicamente una fila de "cajitas", donde cada cajita tiene etiquetas (llaves) con información.

print(output)
#  "Lista" (Los corchetes [])
# "Diccionario" (Las llaves {})


print(output[0]['generated_text'])

[{'generated_text': '😱.\n\nDear Conrado,\n\nI hope this email finds you well. I am writing to express my deepest apologies for the unfortunate events that occurred in your garden recently. I understand that it must'}]
😱.

Dear Conrado,

I hope this email finds you well. I am writing to express my deepest apologies for the unfortunate events that occurred in your garden recently. I understand that it must


In [27]:
## ver cuántos elementos te devolvió el modelo y qué "etiquetas" tienen adentro


In [25]:
# 1. Vemos la cantidad de elementos en la lista
print(f"El modelo devolvió {len(output)} variantes.")

# 2. Investigamos el primer elemento para ver sus etiquetas
if len(output) > 0:
    primer_resultado = output[0]
    print(f"Las etiquetas disponibles son: {list(primer_resultado.keys())}")
    
    # 3. Iteramos para imprimir todo limpio
    print("\n--- CONTENIDO GENERADO ---")
    for i, res in enumerate(output):
        # Usamos .get() por seguridad; si la llave no existe, no rompe
        texto = res.get('generated_text', 'No se encontró la llave generated_text')
        print(f"Variante {i}:\n{texto}\n" + "-"*30)

El modelo devolvió 1 variantes.
Las etiquetas disponibles son: ['generated_text']

--- CONTENIDO GENERADO ---
Variante 0:


Dear Conrado,

I am writing to express my deepest apologies for the tragic incident that occurred in your garden. I am truly sorry for the loss of your beloved pet, and I understand the pain and sorrow
------------------------------


In [29]:
## Architecture of the model . Use this command to print it 


In [28]:
model

Phi3ForCausalLM(
  (model): Phi3Model(
    (embed_tokens): Embedding(32064, 3072, padding_idx=32000)
    (layers): ModuleList(
      (0-31): 32 x Phi3DecoderLayer(
        (self_attn): Phi3Attention(
          (o_proj): Linear(in_features=3072, out_features=3072, bias=False)
          (qkv_proj): Linear(in_features=3072, out_features=9216, bias=False)
        )
        (mlp): Phi3MLP(
          (gate_up_proj): Linear(in_features=3072, out_features=16384, bias=False)
          (down_proj): Linear(in_features=8192, out_features=3072, bias=False)
          (activation_fn): SiLUActivation()
        )
        (input_layernorm): Phi3RMSNorm((3072,), eps=1e-05)
        (post_attention_layernorm): Phi3RMSNorm((3072,), eps=1e-05)
        (resid_attn_dropout): Dropout(p=0.0, inplace=False)
        (resid_mlp_dropout): Dropout(p=0.0, inplace=False)
      )
    )
    (norm): Phi3RMSNorm((3072,), eps=1e-05)
    (rotary_emb): Phi3RotaryEmbedding()
  )
  (lm_head): Linear(in_features=3072, out_featur

In [30]:
model.model

Phi3Model(
  (embed_tokens): Embedding(32064, 3072, padding_idx=32000)
  (layers): ModuleList(
    (0-31): 32 x Phi3DecoderLayer(
      (self_attn): Phi3Attention(
        (o_proj): Linear(in_features=3072, out_features=3072, bias=False)
        (qkv_proj): Linear(in_features=3072, out_features=9216, bias=False)
      )
      (mlp): Phi3MLP(
        (gate_up_proj): Linear(in_features=3072, out_features=16384, bias=False)
        (down_proj): Linear(in_features=8192, out_features=3072, bias=False)
        (activation_fn): SiLUActivation()
      )
      (input_layernorm): Phi3RMSNorm((3072,), eps=1e-05)
      (post_attention_layernorm): Phi3RMSNorm((3072,), eps=1e-05)
      (resid_attn_dropout): Dropout(p=0.0, inplace=False)
      (resid_mlp_dropout): Dropout(p=0.0, inplace=False)
    )
  )
  (norm): Phi3RMSNorm((3072,), eps=1e-05)
  (rotary_emb): Phi3RotaryEmbedding()
)

In [32]:
model.model.layers[0]

Phi3DecoderLayer(
  (self_attn): Phi3Attention(
    (o_proj): Linear(in_features=3072, out_features=3072, bias=False)
    (qkv_proj): Linear(in_features=3072, out_features=9216, bias=False)
  )
  (mlp): Phi3MLP(
    (gate_up_proj): Linear(in_features=3072, out_features=16384, bias=False)
    (down_proj): Linear(in_features=8192, out_features=3072, bias=False)
    (activation_fn): SiLUActivation()
  )
  (input_layernorm): Phi3RMSNorm((3072,), eps=1e-05)
  (post_attention_layernorm): Phi3RMSNorm((3072,), eps=1e-05)
  (resid_attn_dropout): Dropout(p=0.0, inplace=False)
  (resid_mlp_dropout): Dropout(p=0.0, inplace=False)
)